In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import numpy as np

# Load the pair probability table
file_path = "pair_fraction.csv"  # Update with your file path
df = pd.read_csv(file_path, header=None, names=["StrandA", "i", "StrandB", "j", "Probability"])

# Filter out unpaired bases
df_paired = df[df["StrandB"] != -1]

# Create a graph
G = nx.Graph()

# Add edges with probabilities as weights
edges = []
for _, row in df_paired.iterrows():
    G.add_edge(row["i"], row["j"], weight=row["Probability"])
    edges.append((row["i"], row["j"], row["Probability"]))

# Define node positions on a line
nodes = sorted(G.nodes())
pos = {node: (i, 0) for i, node in enumerate(nodes)}

# Split edges into high and low probability
high_prob_edges = [edge for edge in edges if edge[2] > 0.5]
low_prob_edges = [edge for edge in edges if edge[2] <= 0.5]

# Create a figure with three subplots
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(50, 25), gridspec_kw={'height_ratios': [2, 2]})

# Ensure ax is a list-like object
if not isinstance(ax, (list, np.ndarray)):
    ax = [ax]  # Convert to a list if it's a single Axes object

# Plot high-probability arcs in the top subplot (ax[0])
nx.draw_networkx_nodes(G, pos, node_size=0.1, node_color="black", ax=ax[0])

cmap = plt.cm.viridis.reversed()
norm = plt.Normalize(vmin=min(df_paired["Probability"]), vmax=max(df_paired["Probability"]))

# Draw arcs for high-probability edges (above the axis)
for i, j, prob in high_prob_edges:
    x = [pos[i][0], pos[j][0]]
    mid = np.mean(x)
    height = abs(j - i) / len(G.nodes) * 2  # Adjust arc height accordingly
    
    arc_x = np.linspace(x[0], x[1], 100)
    arc_y = height * np.sin(np.linspace(0, np.pi, 100))  # Arcs above the axis
    
    ax[0].plot(arc_x, arc_y, color=cmap(norm(prob)), linewidth=3.5)
      
# Remove all spines, ticks, and grids from both subplots
for axis in ax:
    axis.spines["top"].set_visible(False)
    axis.spines["right"].set_visible(False)
    axis.spines["left"].set_visible(False)
    axis.spines["bottom"].set_visible(False)
    axis.set_xticks([])
    axis.set_yticks([])
    axis.grid(False)

# Add colorbar legend for the top subplot
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax[0], orientation='vertical', fraction=0.02, pad=0.02)
cbar.set_label("Pairing Probability")

# Plot low-probability arcs in the bottom subplot (ax[2])
nx.draw_networkx_nodes(G, pos, node_size=0.1, node_color="black", ax=ax[1])

# Draw arcs for low-probability edges (below the axis)
for i, j, prob in low_prob_edges:
    x = [pos[i][0], pos[j][0]]
    mid = np.mean(x)
    height = abs(j - i) / len(G.nodes) * 2  # Adjust arc height accordingly
    
    arc_x = np.linspace(x[0], x[1], 100)
    arc_y = -height * np.sin(np.linspace(0, np.pi, 100))  # Arcs below the axis
    
    ax[1].plot(arc_x, arc_y, color=cmap(norm(prob)), linewidth=3.5)


# Adjust layout
plt.tight_layout()
plt.savefig("fold_chage_arc_diagram.png", dpi=600, bbox_inches='tight')
plt.show()